In [1]:
import pasttrec_ctrl as ptc

from fst import *
from my_utils import *

import micos_eco
import phidrive
import numpy as np
import time
import os

from IPython.display import Audio
sound_file_end = './blubb.wav'
sound_file_loop = './tock.wav'
Audio(sound_file_loop, autoplay=True)

Welcome to JupyROOT 6.14/04


In [2]:
# measurement parameters individual settings:
id = "id057_trafo_HV1750V"
#threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
#threshold_list = range(10,60,2)
threshold_list = [15]
#scan x position
#X = np.arange(78.5-1.6,78.5+1.6,0.15) # half a drift cell, channel 6
#X = np.arange(78.5-1.6,78.5+4.6,0.1) # full drift cell, channel 6
X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
#X = [80.0] # anode wire position   wire 11 layer 1 
y = 71.0
PHI = np.arange(-4,5,0.5)
#PHI = [0]
acq_time = 60  #  sec
sync_spill_end = False
#-----------------------

#predefined systematic measurements for input stage comparison:
measurement_setting = 4  #1 or 2 ... or 7
# 2x  x-scan = 22 steps each  0.1 mm , MDC HV  1780V
# 4x threshold scan  for MDC HV  1780V, 1750V, 1700V, 1650V
# 4x night selftraking 2h for each threshold 15, 30, 50, 10,  MDC HV  1750V
if (measurement_setting == 1):
    id = "id081_NOBEAM_HV1750V"
    X = np.arange(78.5-1.6,78.5+1.6,0.15) # half a drift cell, channel 6
    y = 71.0
    threshold_list = [15,30]
    PHI = [0]
    acq_time = 71  #  sec
    sync_spill_end = False
elif  (measurement_setting == 2):
    id = "id082_NOBEAM_HV1780V"
    X = np.arange(78.5-1.6,78.5+4.6,0.1) # full drift cell, channel 6
    y = 71.0
    threshold_list = [15]
    PHI = [0]
    acq_time = 71  #  sec
    sync_spill_end = False
elif  (measurement_setting == 3):
    id = "id083_NOBEAM_HV1780V"
    X = [78.5] # near UFSD3  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
    PHI = [0]
    acq_time = 71  #  sec
    sync_spill_end = False    
elif  (measurement_setting == 4):
    id = "id090_NOBEAM_digitalPureACsolderground_HV1750V"
    X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
    PHI = [0]
    acq_time = 71  #  sec
    sync_spill_end = False    
elif  (measurement_setting == 5):
    id = "id055_TRAFO_HV1700V"
    X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
    PHI = [0]
    acq_time = 60  #  sec
    sync_spill_end = False
elif  (measurement_setting == 6):
    id = "id056_TRAFO_HV1650V"
    X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
    PHI = [0]
    acq_time = 60  #  sec
    sync_spill_end = False
elif  (measurement_setting == 7):
    id = "id042_TRAFO_HV1750V"
    X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [15, 30, 50, 10]
    PHI = [0]
    acq_time = 2*60*60  #  sec
    sync_spill_end = False   
    


if False:
    id = "id061_pAC_HV1750V"
    X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
    PHI = [0]
    acq_time = 60  #  sec
    sync_spill_end = False      
    
    id = "id062_pAC_HV1750V"
    X = np.arange(78.5-1.6,78.5+1.6,0.15) # half a drift cell, channel 6
    y = 71.0
    threshold_list = [15]
    PHI = [0]
    acq_time = 60  #  sec
    sync_spill_end = False  
    
    
    id = "id071_pAC_ana_board_HV1750V"
    X = [78.5] # near UFSD  position of selftracking on wire 11 layer 1 
    y = 71.0
    threshold_list = [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
    PHI = [0]
    acq_time = 60  #  sec
    sync_spill_end = False    



# end of parameters to edit ############
print(id)
print("number of points to measure = {:d} ".format(len(X)*len(PHI)*len(threshold_list)) )
print("thresholds ",end="")
print(threshold_list)
print("phi angles ",end="")
print(PHI)
print("x positions ",end="")
print(X)
print("y position = {:3.2f} ".format(y))

full_loop_time = acq_time
if sync_spill_end:
    acq_time = 18
    full_loop_time = 78 # full spill
minutes = len(threshold_list)*(full_loop_time)*len(X)*len(PHI)/60
hours = minutes/60
print("estimated scan duration {:3.1f} minutes or {:3.2f} hours".format(minutes,hours))

import datetime
now = datetime.datetime.now()
print("Starting now {:s} ".format(str(now)))

id090_NOBEAM_digitalPureACsolderground_HV1750V
number of points to measure = 10 
thresholds [6, 8, 10, 12, 15, 20, 25, 30, 40, 50]
phi angles [0]
x positions [78.5]
y position = 71.00 
estimated scan duration 11.8 minutes or 0.20 hours
Starting now 2019-11-10 15:36:34.266198 


In [3]:
#trigger_ufsd()
trigger_scinti()
for x in X:
 for phi in PHI:
  for thresh in threshold_list:
    print("")
    print("##############################")
    print("##### setting threshold {:d}".format(thresh))
    print("##############################")
    print("")
    print("##############################")
    print("##### move x {:f}".format(x))
    print("##############################")
    print("")
    print("##############################")
    print("##### rotate phi {:f}".format(phi))
    print("##############################")
    micos_eco.move(x=x,y=y)
    if (phi != phidrive.get_angle() ): 
        phidrive.set_angle(phi)
    ptc.init_active_boards()
    ptc.set_threshold_for_board_by_name("0009",thresh)
    ptc.set_threshold_for_board_by_name("0014",thresh)
    if acq_time < 70 :
      wait_for_spill_break(threshold_rate=1000)
      wait_for_spill(threshold_rate=1600)
    if sync_spill_end :
      print("wait 45s until last 10s of spill (low intensity)")
      time.sleep(45)
    print("##### start taking data ######")
    take_raw_data(time=acq_time,label="{:s}_thr{:d}_{:d}s_x{:3.3f}_phi{:3.2f}".format(id,thresh,acq_time,x,phi)) 
    print("##### stopped data taking ######")
    os.system("/workdir/tock86.sh")


##############################
##### setting threshold 6
##############################

##############################
##### move x 78.500000
##############################

##############################
##### rotate phi 0.000000
##############################
(78.5, 71.0, 0.0)
done
init board 0009
set baselines of the following channels
[16, 17, 18, 19, 20, 21, 22, 23, 24]
to the following values
[1.0, 5.0, -1.0, 7.0, 5.0, 4.0, 3.0, 7.0, 6.0]
set baselines of the following channels
[24, 25, 26, 27, 28, 29, 30, 31]
to the following values
[6.0, -0.0, 1.0, 0.0, -4.0, 3.0, 2.0, 4.0]
init board 0014
set baselines of the following channels
[0, 1, 2, 3, 4, 5, 6, 7, 8]
to the following values
[3.0, 2.0, 7.0, 8.0, 2.0, 5.0, 3.0, 7.0, 9.0]
set baselines of the following channels
[8, 9, 10, 11, 12, 13, 14, 15]
to the following values
[9.0, 8.0, 3.0, 8.0, 5.0, 8.0, 4.0, 0.0]
##### start taking data ######
##### stopped data taking ######

##############################
##### setting threshold

set baselines of the following channels
[8, 9, 10, 11, 12, 13, 14, 15]
to the following values
[9.0, 8.0, 3.0, 8.0, 5.0, 8.0, 4.0, 0.0]
##### start taking data ######
##### stopped data taking ######

##############################
##### setting threshold 50
##############################

##############################
##### move x 78.500000
##############################

##############################
##### rotate phi 0.000000
##############################
(78.5, 71.0, 0.0)
done
init board 0009
set baselines of the following channels
[16, 17, 18, 19, 20, 21, 22, 23, 24]
to the following values
[1.0, 5.0, -1.0, 7.0, 5.0, 4.0, 3.0, 7.0, 6.0]
set baselines of the following channels
[24, 25, 26, 27, 28, 29, 30, 31]
to the following values
[6.0, -0.0, 1.0, 0.0, -4.0, 3.0, 2.0, 4.0]
init board 0014
set baselines of the following channels
[0, 1, 2, 3, 4, 5, 6, 7, 8]
to the following values
[3.0, 2.0, 7.0, 8.0, 2.0, 5.0, 3.0, 7.0, 9.0]
set baselines of the following channels
[8, 9, 10, 11,

rm: cannot remove '/workdir/data/*.hld': No such file or directory
dabc_exe: no process found
Playing WAVE '/home/hadaq/tock.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo
rm: cannot remove '/workdir/data/*.hld': No such file or directory
dabc_exe: no process found
Playing WAVE '/home/hadaq/tock.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo
rm: cannot remove '/workdir/data/*.hld': No such file or directory
dabc_exe: no process found
Playing WAVE '/home/hadaq/tock.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo
rm: cannot remove '/workdir/data/*.hld': No such file or directory
dabc_exe: no process found
Playing WAVE '/home/hadaq/tock.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo
rm: cannot remove '/workdir/data/*.hld': No such file or directory
dabc_exe: no process found
Playing WAVE '/home/hadaq/tock.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo
rm: cannot remove '/workdir/data/*.hld': No such file or directory
dabc_exe: no proce

In [4]:
archive(label="{:s}_{:d}s".format(id,acq_time))


Audio(sound_file_end, autoplay=True)

archived to ./archive/2019-11-10_15-49-32_pasttrec_scan_universal_id090_NOBEAM_digitalPureACsolderground_HV1750V_71s
